In [63]:
import requests
import urllib3
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import time
from datetime import date
import seaborn as sns
import matplotlib.pyplot as plt
#import stravalib

import polyline

import folium

pd.set_option("display.max_columns", 100)


Interesting link: https://towardsdatascience.com/using-the-strava-api-and-pandas-to-explore-your-activity-data-d94901d9bfde
Strava API forum: https://groups.google.com/g/strava-api

In [64]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
payload = {
    'client_id': "75483",
    'client_secret': 'd7d1e5483201559ddf62c677ac12f6cf7847c5ae',
    'refresh_token': '75994cf8182a3b6a956390b5a4da3eba764f1d85',
    'grant_type': "refresh_token",
    'f': 'json',
}

In [65]:
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

Requesting Token...

Access Token = b3cf17818d406b57ee795c978b8918c84271e4ac



In [66]:
activites_url = "https://www.strava.com/api/v3/athlete/activities"

dataset=[]
pages_count=1

while pages_count>0:
    header = {'Authorization': 'Bearer ' + access_token}
    param = {'per_page': 200, 'page': pages_count}
    my_dataset=requests.get(activites_url, headers=header, params=param).json()
    if my_dataset==[]:
        break
    elif my_dataset!=[]:
        dataset.append(my_dataset)
        pages_count+=1
dataset = [j for i in dataset for j in i]


In [67]:
activities = pd.json_normalize(dataset)
activities

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,workout_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_cadence,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_watts,kilojoules,device_watts,average_temp
0,2,Afternoon Run,17071.1,5450,5589,181.6,Run,0.0,6965765652,2022-04-11T15:38:54Z,2022-04-11T17:38:54Z,(GMT+01:00) Europe/Madrid,7200.0,None,None,None,10,5,0,1,0,False,False,False,False,followers_only,False,None,"[40.39984239265323, -3.7644396163523197]","[40.399888744577765, -3.7645990401506424]",3.132,3.952,83.3,True,159.9,181.0,False,True,694.4,593.1,7412875786,7412875786,garmin_push_8620446887,False,5,0,False,183.0,30195369,1,a6965765652,_rquFvf~U?H[d@kBt@_@ZGPxCdDr@tA|@|@Pl@Cr@cArJB...,2,NaN,NaN,NaN,NaN
1,2,Evening Swim,1200.0,1521,1747,0.0,Swim,NaN,6965179647,2022-04-08T17:46:53Z,2022-04-08T19:46:53Z,(GMT+01:00) Europe/Madrid,7200.0,None,None,None,0,0,0,1,0,True,False,False,True,only_me,False,None,[],[],0.789,7.339,24.6,False,NaN,NaN,False,False,NaN,NaN,7412242899,7412242899,garmin_push_8619734695,False,0,0,False,NaN,30195369,1,a6965179647,None,2,NaN,NaN,NaN,NaN
2,2,Lunch Run,6250.3,2157,2244,116.8,Run,NaN,6944305092,2022-04-07T10:35:51Z,2022-04-07T12:35:51Z,(GMT+01:00) Europe/Madrid,7200.0,None,None,None,0,0,0,1,0,False,False,False,True,only_me,False,None,"[40.392842246219516, -3.759979773312807]","[40.39287082850933, -3.760444885119796]",2.898,3.793,81.7,True,160.4,183.0,False,True,665.4,642.2,7389618864,7389618864,garmin_push_8595885335,False,0,0,False,75.0,30195369,1,a6944305092,gfpuFzj}Ud@[z@iAb@E~@t@d@DxDMhAd@d@Nd@Hd@C`@Ub...,2,NaN,NaN,NaN,NaN
3,2,Afternoon Ride,24208.3,5585,6763,461.1,Ride,10.0,6941196818,2022-04-06T15:33:17Z,2022-04-06T17:33:17Z,(GMT+01:00) Europe/Madrid,7200.0,None,None,None,15,7,0,1,0,False,False,False,False,followers_only,False,None,"[40.394009007140994, -3.762325281277299]","[40.39428435266018, -3.762727864086628]",4.335,10.481,NaN,True,146.4,185.0,False,True,693.6,614.8,7386266995,7386266995,garmin_push_8592346689,False,6,0,False,104.0,30195369,1,a6941196818,ompuFpy}UgHvIsEtEqA|@eC}CBs@aAqAq@eDw@ZuAgBy@t...,2,75.1,419.5,False,NaN
4,2,Afternoon Ride,63727.8,9385,10001,472.0,Ride,10.0,6931170596,2022-04-04T14:47:38Z,2022-04-04T16:47:38Z,(GMT+01:00) Europe/Madrid,7200.0,None,None,None,193,9,0,1,0,False,False,False,False,followers_only,False,None,"[40.397547679021955, -3.7645368464291096]","[40.39496990852058, -3.7595066986978054]",6.790,13.797,NaN,True,148.8,176.0,False,True,764.9,591.6,7375479636,7375479636,garmin_push_8580750866,False,30,0,False,200.0,30195369,1,a6931170596,scquFjg~UiAeB}AwGiAcJXmEvC_G^qCkDMcDlEU~HtBnMk...,2,93.0,872.6,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,2,Ciclismo al anochecer,47446.6,7447,7632,295.5,Ride,10.0,1578340888,2018-05-17T16:01:00Z,2018-05-17T18:01:00Z,(GMT+01:00) Europe/Madrid,7200.0,None,None,None,38,0,0,1,0,False,False,False,False,followers_only,False,None,"[42.248587, 2.958732]","[42.248741, 2.958242]",6.371,13.400,NaN,False,NaN,NaN,False,False,123.0,0.9,1696975141,1696975141,f5514f296dc4626b4a6dc25b51f5d0e5,False,9,0,False,NaN,30195369,1,a1578340888,qzz`Gat}Pk@nEHnC~ATn@x@rArDg@xB`DxDvA~@nFm@hFn...,2,NaN,NaN,False,NaN
457,2,Estirando piernas afterwork,25017.7,6163,6294,249.4,Ride,10.0,1576366408,2018-05-16T16:42:20Z,2018-05-16T18:42:20

In [68]:
#Create new dataframe with only columns I care about
cols =['name',
        'id',
        'type',
        'map.summary_polyline']

activities = activities[cols]
activities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  461 non-null    object
 1   id                    461 non-null    int64 
 2   type                  461 non-null    object
 3   map.summary_polyline  375 non-null    object
dtypes: int64(1), object(3)
memory usage: 14.5+ KB


In [69]:
activities['type'].value_counts()

Ride              174
Run               143
Swim               91
Hike               50
Workout             2
BackcountrySki      1
Name: type, dtype: int64

In [70]:
rides = activities[activities['type']=='Ride']
rides

,name,id,type,map.summary_polyline
3,Afternoon Ride,6941196818,Ride,ompuFpy}UgHvIsEtEqA|@eC}CBs@aAqAq@eDw@ZuAgBy@t...
4,Afternoon Ride,6931170596,Ride,scquFjg~UiAeB}AwGiAcJXmEvC_G^qCkDMcDlEU~HtBnMk...
18,Afternoon Ride,6798524273,Ride,ulpuFh`~Ug@|FeEjCg@BqA_F_DjEoAd@}CqDN_@_CoFk@D...
28,Afternoon Ride,6702635727,Ride,qlpuFx_~U@eE`D}Ev@yPiRqAkGyCcBfC}CUaDnE[tCPfH`...
45,Lunch Ride,6532557667,Ride,ompuFr`~UM_E_DsGFkIaISsBwA}BsEqB_@u@JcDhEWrIr@...
...,...,...,...,...
455,St Pere de Rodes,1581604812,Ride,sm|`G}z`Q{Bf@c@uAq@SgS|AcAv@cA]\OmD^{DoVqDyc@y...
456,Ciclismo al anochecer,1578340888,Ride,qzz`Gat}Pk@nEHnC~ATn@x@rArDg@xB`DxDvA~@nFm@hFn...
457,Estirando piernas afterwork,1576366408,Ride,siy`Gwv~Pd@FTOvA{EVkBAe@JoCEaBJkB\qBf@sBfBuGDe...
458,Ciclismo por la tarde,1571859965,Ride,_y{`Gs~aQ_HoDiBi@o@k@u@LcDUi@sD_AuCAgA]kA_CgOg...


In [71]:
ride_id = 6941196818
latlon=polyline.decode(rides.loc[rides['id'] == ride_id, 'map.summary_polyline'].iloc[0])

In [72]:
mapit = None
for coord in latlon:
    mapit = folium.Map( location=[ coord[0], coord[1] ] )

folium.PolyLine(latlon,
                color='red',
                weight=3,
                opacity=0.8).add_to(mapit)

mapit.save( 'map.html')

mapit

### Get the maps of the routes
https://developers.google.com/maps/documentation/utilities/polylineutility